In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.7 MB/s eta 0:00:00


In [ ]:
! pip install datasets transformers seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [ ]:
import transformers

print(transformers.__version__)

4.28.1


# Fine-tuning a model on a token classification task

In [ ]:
!wget https://dl.fbaipublicfiles.com/biolm/RoBERTa-large-PM-M3-Voc-hf.tar.gz

--2023-04-21 16:35:21--  https://dl.fbaipublicfiles.com/biolm/RoBERTa-large-PM-M3-Voc-hf.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.155.68.108, 18.155.68.81, 18.155.68.27, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.155.68.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 839592355 (801M) [application/gzip]
Saving to: ‘RoBERTa-large-PM-M3-Voc-hf.tar.gz’

RoBERTa-large-PM-M3 100%[===================>] 800.70M  24.3MB/s    in 36s     

2023-04-21 16:35:57 (22.4 MB/s) - ‘RoBERTa-large-PM-M3-Voc-hf.tar.gz’ saved [839592355/839592355]



In [ ]:
!tar -xf RoBERTa-large-PM-M3-Voc-hf.tar.gz

In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "/content/RoBERTa-large-PM-M3-Voc/RoBERTa-large-PM-M3-Voc-hf"
batch_size = 16

## Loading the dataset

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
datasets = load_dataset("ktgiahieu/maccrobat2018_2020")
datasets = datasets['train'].train_test_split(test_size=0.1)

  0%|          | 0/1 [00:00<?, ?it/s]

The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 360
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 40
    })
})

In [ ]:
from datasets import Dataset, ClassLabel, Value, DatasetDict

# Define the ClassLabel feature with the label names
label_list = ['B-Activity', 'B-Administration', 'B-Age', 'B-Area', 'B-Biological_attribute', 'B-Biological_structure', 'B-Clinical_event', 'B-Color', 'B-Coreference', 'B-Date', 'B-Detailed_description', 'B-Diagnostic_procedure', 'B-Disease_disorder', 'B-Distance', 'B-Dosage', 'B-Duration', 'B-Family_history', 'B-Frequency', 'B-Height', 'B-History', 'B-Lab_value', 'B-Mass', 'B-Medication', 'B-Nonbiological_location', 'B-Occupation', 'B-Other_entity', 'B-Other_event', 'B-Outcome', 'B-Personal_background', 'B-Qualitative_concept', 'B-Quantitative_concept', 'B-Severity', 'B-Sex', 'B-Shape', 'B-Sign_symptom', 'B-Subject', 'B-Texture', 'B-Therapeutic_procedure', 'B-Time', 'B-Volume', 'B-Weight', 'I-Activity', 'I-Administration', 'I-Age', 'I-Area', 'I-Biological_attribute', 'I-Biological_structure', 'I-Clinical_event', 'I-Color', 'I-Coreference', 'I-Date', 'I-Detailed_description', 'I-Diagnostic_procedure', 'I-Disease_disorder', 'I-Distance', 'I-Dosage', 'I-Duration', 'I-Family_history', 'I-Frequency', 'I-Height', 'I-History', 'I-Lab_value', 'I-Mass', 'I-Medication', 'I-Nonbiological_location', 'I-Occupation', 'I-Other_entity', 'I-Other_event', 'I-Outcome', 'I-Personal_background', 'I-Qualitative_concept', 'I-Quantitative_concept', 'I-Severity', 'I-Shape', 'I-Sign_symptom', 'I-Subject', 'I-Texture', 'I-Therapeutic_procedure', 'I-Time', 'I-Volume', 'I-Weight', 'O']
label = ClassLabel(names=label_list)
# Define a new feature with the numeric labels
numeric_labels_feature = Value("int32")

In [ ]:
datasets = datasets.map(lambda example: {"tokens": example['tokens'],
    "tags": example["tags"],
    "numeric_tags": [label.encode_example(x) for x in example["tags"]],
})

Map:   0%|          | 0/360 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags', 'numeric_tags'],
        num_rows: 360
    })
    test: Dataset({
        features: ['tokens', 'tags', 'numeric_tags'],
        num_rows: 40
    })
})

In [ ]:
# label_list = datasets["train"].features[f"{task}_tags"].feature.names
# label_list

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,tokens,tags,numeric_tags
0,"[A, 31, -, year, -, old, man, developed, diabetes, insipidus, with, urine, volume, up, to, 10, to, 20, , L, every, 24, , hours, in, 2003, ., \n, Four, years, later, ,, he, complained, of, fatigue, ,, anorexia, ,, jaundice, and, pruritus, ,, and, a, symptomatic, occipital, mass, ., \n, Laboratory, tests, showed, an, abnormal, liver, enzyme, (, Table, ​1, ), ,, the, patient, was, negative, for, hepatitis, viruses, ., \n, As, shown, in, Fig.1, ,, abdominal, MRI, showed, multiple, low, -, density, lesions, in, the, liver, on, the, T1, -, weighted, image, and, obvious, expansion, of, the, intrahepatic, bile, ...]","[O, B-Age, I-Age, I-Age, I-Age, I-Age, B-Sex, O, B-Disease_disorder, I-Disease_disorder, O, B-Diagnostic_procedure, I-Diagnostic_procedure, O, O, B-Volume, I-Volume, I-Volume, I-Volume, I-Volume, B-Frequency, I-Frequency, I-Frequency, I-Frequency, O, B-Date, O, O, B-Date, I-Date, I-Date, O, O, O, O, B-Sign_symptom, O, B-Sign_symptom, O, B-Sign_symptom, O, B-Sign_symptom, O, O, O, O, O, B-Sign_symptom, O, O, B-Diagnostic_procedure, I-Diagnostic_procedure, O, O, O, B-Diagnostic_procedure, I-Diagnostic_procedure, O, O, O, O, O, O, O, O, O, O, B-Diagnostic_procedure, I-Diagnostic_procedure, O, O, O, O, O, O, O, O, B-Diagnostic_procedure, O, O, O, O, O, B-Sign_symptom, O, O, B-Biological_structure, O, O, B-Diagnostic_procedure, I-Diagnostic_procedure, I-Diagnostic_procedure, I-Diagnostic_procedure, O, O, O, O, O, O, O, ...]","[81, 2, 43, 43, 43, 43, 32, 81, 12, 53, 81, 11, 52, 81, 81, 39, 79, 79, 79, 79, 17, 58, 58, 58, 81, 9, 81, 81, 9, 50, 50, 81, 81, 81, 81, 34, 81, 34, 81, 34, 81, 34, 81, 81, 81, 81, 81, 34, 81, 81, 11, 52, 81, 81, 81, 11, 52, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 11, 52, 81, 81, 81, 81, 81, 81, 81, 81, 11, 81, 81, 81, 81, 81, 34, 81, 81, 5, 81, 81, 11, 52, 52, 52, 81, 81, 81, 81, 81, 81, 81, ...]"
1,"[A, 36-, year-, old, white, woman, at, 40, week, of, gestation, was, admitted, to, a, local, obstetric, clinic, for, elective, repeated, cesarean, delivery, due, to, slanting, position, of, the, fetus, ., \n, Both, her, previous, and, current, pregnancy, were, uncomplicated, ., \n, Her, past, medical, history, was, unremarkable, and, no, family, history, of, cardiac, disease, was, reported, ., \n, The, caesarean, delivery, was, performed, following, the, administration, of, standard, spinal, anesthesia, ., \n, During, surgery, the, patient, became, hypotensive, and, ephedrine, was, injected, to, maintain, her, blood, pressure, in, the, normal, range, ., \n, Three, hours, after, delivery, of, healthy, ...]","[O, B-Age, I-Age, I-Age, B-Personal_background, B-Sex, O, O, O, O, O, O, B-Clinical_event, O, O, B-Nonbiological_location, I-Nonbiological_location, I-Nonbiological_location, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-History, I-History, I-History, I-History, I-History, I-History, O, O, O, B-History, I-History, I-History, I-History, I-History, O, B-Family_history, I-Family_history, I-Family_history, I-Family_history, I-Family_history, I-Family_history, I-Family_history, I-Family_history, O, O, O, B-Therapeutic_procedure, I-Therapeutic_procedure, O, O, O, O, O, O, O, O, B-Medication, O, O, O, O, O, O, O, B-Sign_symptom, O, B-Medication, O, B-Administration, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...]","[81, 2, 43, 43, 28, 32, 81, 81, 81, 81, 81, 81, 6, 81, 81, 23, 64, 64, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 19, 60, 60, 60, 60, 60, 81, 81, 81, 19, 60, 60, 60, 60, 81, 16, 57, 57, 57, 57, 57, 57, 57, 81, 81, 81, 37, 77, 81, 81, 81, 81, 81, 81, 81, 81, 22, 81, 81, 81, 81, 81, 81, 81, 34, 81, 22, 81, 1, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, ...]"
2,"[A, 21, -, year, -, old, male, presented, with, a, 2, -, year, history, of, progressive, shortness, of, breath, on, exertion, and, dry, cough, ., \n, At, physical, examination, ,, auscultation, of, the, lungs, has, revealed, random, wheezes, and, coarse, crack

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [0, 427, 24017, 3267, 491, 345, 818, 17812, 9490, 860, 6786, 2245, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
example = datasets["train"][4]
print(example["tokens"])

['The', 'patient', ',', 'a', '4', '-', 'year', '-', 'old', 'boy', ',', 'was', 'diagnosed', 'as', 'having', 'Blau', 'syndrome', 'based', 'on', 'the', 'manifestation', 'of', 'typical', 'clinical', 'features', '(', 'ankle', 'and', 'wrist', 'arthritis', '/', 'tenosynovitis', ',', 'diffuse', 'eczematous', 'rash', ',', 'and', 'uveitis', ')', ',', 'histologic', 'evidence', 'of', 'noncaseating', 'granulomas', ',', 'and', 'a', 'heterozygous', 'NOD2', 'mutation', '(', 'p.', 'R334W', ')', '.', '\n', 'Ocular', 'involvement', 'was', 'initially', 'controlled', 'by', 'topical', 'and', 'oral', 'corticosteroids', ',', 'but', 'over', 'the', 'years', 'visual', 'impairment', 'progressed', '.', '\n', 'Other', 'manifestations', 'of', 'Blau', 'syndrome', '(', 'arthritis', 'and', 'rash', ')', 'subsided', 'over', 'time', '.', '\n', 'Bilateral', 'panuveitis', 'progressed', 'after', 'age', '5', 'years', ',', 'and', 'was', 'initially', 'treated', 'with', 'methotrexate', '.', '\n', 'However', ',', 'ocular', 'infla

In [ ]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['<s>', 'ĠThe', 'Ġpatient', 'Ġ,', 'Ġa', 'Ġ4', 'Ġ-', 'Ġyear', 'Ġ-', 'Ġold', 'Ġboy', 'Ġ,', 'Ġwas', 'Ġdiagnosed', 'Ġas', 'Ġhaving', 'ĠBl', 'au', 'Ġsyndrome', 'Ġbased', 'Ġon', 'Ġthe', 'Ġmanifestation', 'Ġof', 'Ġtypical', 'Ġclinical', 'Ġfeatures', 'Ġ(', 'Ġankle', 'Ġand', 'Ġwrist', 'Ġarthritis', 'Ġ/', 'Ġten', 'osyn', 'ovi', 'tis', 'Ġ,', 'Ġdiffuse', 'Ġec', 'z', 'ematous', 'Ġrash', 'Ġ,', 'Ġand', 'Ġuveitis', 'Ġ)', 'Ġ,', 'Ġhistologic', 'Ġevidence', 'Ġof', 'Ġnon', 'case', 'ating', 'Ġgranulomas', 'Ġ,', 'Ġand', 'Ġa', 'Ġheterozygous', 'ĠNOD', '2', 'Ġmutation', 'Ġ(', 'Ġp', '.', 'ĠR', '334', 'W', 'Ġ)', 'Ġ.', 'Ġ', 'Ċ', 'ĠOc', 'ular', 'Ġinvolvement', 'Ġwas', 'Ġinitially', 'Ġcontrolled', 'Ġby', 'Ġtopical', 'Ġand', 'Ġoral', 'Ġcorticosteroids', 'Ġ,', 'Ġbut', 'Ġover', 'Ġthe', 'Ġyears', 'Ġvisual', 'Ġimpairment', 'Ġprogressed', 'Ġ.', 'Ġ', 'Ċ', 'ĠOther', 'Ġmanifestations', 'Ġof', 'ĠBl', 'au', 'Ġsyndrome', 'Ġ(', 'Ġarthritis', 'Ġand', 'Ġrash', 'Ġ)', 'Ġsubsided', 'Ġover', 'Ġtime', 'Ġ.', 'Ġ', 'Ċ', 'ĠB', 'ilateral'

In [ ]:
len(example[f"numeric_tags"]), len(tokenized_input["input_ids"])

(428, 483)

In [ ]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 31, 31, 31, 32, 33, 34, 34, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 44, 44, 45, 46, 47, 48, 49, 50, 50, 51, 52, 53, 53, 54, 54, 54, 55, 56, 57, 57, 58, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 77, 78, 79, 80, 81, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 92, 93, 93, 94, 94, 94, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 192, 193, 193, 194, 195, 196, 197, 198, 198,

In [ ]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"numeric_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

483 483


In [ ]:
label_all_tokens = True

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, max_length=512)

    labels = []
    for i, label in enumerate(examples[f"numeric_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[0, 362, 3607, 1878, 2520, 1878, 4013, 779, 343, 5476, 302, 875, 2144, 329, 4612, 996, 358, 343, 1730, 841, 3041, 382, 12022, 42168, 11478, 28411, 6255, 301, 41418, 6970, 2245, 224, 202, 3759, 609, 2186, 2819, 343, 1281, 3267, 4535, 2385, 6878, 2523, 382, 6517, 9111, 2245, 224, 202, 20963, 3550, 653, 925, 261, 1779, 640, 281, 21526, 281, 4183, 17, 22, 4382, 10184, 4145, 301, 1222, 1648, 13588, 541, 1878, 1430, 4382, 10184, 4145, 6970, 3267, 720, 689, 691, 12862, 3267, 530, 1408, 691, 1294, 2791, 3267, 853, 340, 3267, 41217, 3319, 301, 16648, 6970, 290, 9885, 3319, 971, 1878, 811, 301, 2571, 1311, 1878, 811, 6970, 2245, 224, 202, 41418, 1720, 261, 545, 1878, 3203, 8072, 1560, 1468, 696, 2000, 358, 343, 7894, 281, 991, 49268, 14441, 276, 270, 1970, 13952, 281, 270, 4612, 39021, 2245, 224, 202, 362, 5172, 3352, 281, 8853, 5159, 2312, 2413, 270, 1401, 281, 10683, 6875, 11164, 8226, 2245, 224, 202, 20463, 47809, 39892, 4825, 5985, 8287, 261, 4612, 2000, 324, 4478, 22225, 276,

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/360 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))



Some weights of the model checkpoint at /content/RoBERTa-large-PM-M3-Voc/RoBERTa-large-PM-M3-Voc-hf were not used when initializing RobertaForTokenClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at /content/RoBERTa-large-PM-M3-Voc/RoBERTa-large-PM-M3-Voc-hf and are newly initi

In [ ]:
datasets['train'].shape

(360, 3)

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
    lr_scheduler_type='cosine',

)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
metric = load_metric("seqeval")

<ipython-input-29-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [ ]:
labels = [label_list[i] for i in example[f"numeric_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'Administration': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'Age': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'Date': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'Detailed_description': {'precision': 1.0,
  'recall': 1.0,
  'f1': 1.0,
  'number': 6},
 'Diagnostic_procedure': {'precision': 1.0,
  'recall': 1.0,
  'f1': 1.0,
  'number': 4},
 'Disease_disorder': {'precision': 1.0,
  'recall': 1.0,
  'f1': 1.0,
  'number': 15},
 'Dosage': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 8},
 'Duration': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3},
 'Frequency': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'Lab_value': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'Medication': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 15},
 'Severity': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'Sex': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'Sign_symptom': {

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/ktgiahieu/RoBERTa-large-PM-M3-Voc-hf-finetuned-ner into local empty directory.


In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.276480,1.000000,0.002057,0.004105,0.709037
2,No log,0.823640,0.454307,0.356188,0.399308,0.775029
3,No log,0.619954,0.509612,0.536167,0.522553,0.804592
4,No log,0.574816,0.519608,0.635927,0.571913,0.812821
5,No log,0.568968,0.504833,0.698320,0.586018,0.815075
6,No log,0.461697,0.585149,0.699691,0.637315,0.855698
7,No log,0.455794,0.585240,0.736716,0.652299,0.858528
8,No log,0.399510,0.624077,0.753514,0.682715,0.883216
9,No log,0.367894,0.644614,0.754885,0.695405,0.892599
10,No log,0.333234,0.686681,0.772369,0.727009,0.905913


/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=460, training_loss=0.4281462627908458, metrics={'train_runtime': 538.3623, 'train_samples_per_second': 13.374, 'train_steps_per_second': 0.854, 'total_flos': 6688504052121600.0, 'train_loss': 0.4281462627908458, 'epoch': 20.0})

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [ ]:
trainer.evaluate()

{'eval_loss': 0.299665629863739,
 'eval_precision': 0.7370347773032336,
 'eval_recall': 0.8282482002056908,
 'eval_f1': 0.7799838579499596,
 'eval_accuracy': 0.9265646294160813,
 'eval_runtime': 1.356,
 'eval_samples_per_second': 29.5,
 'eval_steps_per_second': 2.212,
 'epoch': 20.0}

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'Activity': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 5},
 'Administration': {'precision': 1.0,
  'recall': 0.4444444444444444,
  'f1': 0.6153846153846153,
  'number': 9},
 'Age': {'precision': 0.9285714285714286,
  'recall': 0.9069767441860465,
  'f1': 0.9176470588235294,
  'number': 43},
 'Area': {'precision': 0.625,
  'recall': 0.8333333333333334,
  'f1': 0.7142857142857143,
  'number': 12},
 'Biological_structure': {'precision': 0.6328671328671329,
  'recall': 0.7973568281938326,
  'f1': 0.7056530214424951,
  'number': 227},
 'Clinical_event': {'precision': 0.5294117647058824,
  'recall': 0.9,
  'f1': 0.6666666666666667,
  'number': 50},
 'Color': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1},
 'Coreference': {'precision': 0.38095238095238093,
  'recall': 0.21052631578947367,
  'f1': 0.2711864406779661,
  'number': 38},
 'Date': {'precision': 0.6179775280898876,
  'recall': 0.873015873015873,
  'f1': 0.7236842105263157,
  'number': 63},
 'Detailed_descript

In [ ]:
!pip install transformers


In [ ]:
import transformers

In [ ]:
!pip install tensorflow==2.15

In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("d4data/biomedical-ner-all")
model = AutoModelForTokenClassification.from_pretrained("d4data/biomedical-ner-all")

pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")
pipe("""Sure, here are two lines of a medical report:Patient Name: John Doe, Age: 45, Blood Pressure: 120/80 mmHg, Heart Rate: 72 bpm, Temperature: 98.6°F, Respiratory Rate: 16 breaths/min. Diagnosis: The patient presents with symptoms of upper respiratory infection, including cough, nasal congestion, and low-grade fever. Suspected viral etiology. Treatment plan includes rest, hydration, and over-the-counter cold medications. Follow-up appointment scheduled in 3 days for reassessment.""")


[{'entity_group': 'Diagnostic_procedure',
  'score': 0.99622244,
  'word': 'blood pressure',
  'start': 78,
  'end': 92},
 {'entity_group': 'Lab_value',
  'score': 0.99992347,
  'word': '120',
  'start': 94,
  'end': 97},
 {'entity_group': 'Lab_value',
  'score': 0.718937,
  'word': '80',
  'start': 98,
  'end': 100},
 {'entity_group': 'Diagnostic_procedure',
  'score': 0.96984684,
  'word': 'heart rate',
  'start': 107,
  'end': 117},
 {'entity_group': 'Lab_value',
  'score': 0.99993455,
  'word': '72',
  'start': 119,
  'end': 121},
 {'entity_group': 'Diagnostic_procedure',
  'score': 0.96307063,
  'word': 'temperature',
  'start': 127,
  'end': 138},
 {'entity_group': 'Lab_value',
  'score': 0.9998766,
  'word': '98',
  'start': 140,
  'end': 142},
 {'entity_group': 'Lab_value',
  'score': 0.99049497,
  'word': '6°',
  'start': 143,
  'end': 145},
 {'entity_group': 'Diagnostic_procedure',
  'score': 0.9916157,
  'word': 'respiratory rate',
  'start': 148,
  'end': 164},
 {'entity_gr

In [ ]:
input_text = "Sure, here are two lines of a medical report:Patient Name: John Doe, Age: 45, Blood Pressure: 120/80 mmHg, Heart Rate: 72 bpm, Temperature: 98.6°F, Respiratory Rate: 16 breaths/min. Diagnosis: The patient presents with symptoms of upper respiratory infection, including cough, nasal congestion, and low-grade fever. Suspected viral etiology. Treatment plan includes rest, hydration, and over-the-counter cold medications. Follow-up appointment scheduled in 3 days for reassessment.."
entities = pipe(input_text)

# Create a list to store the tokens with highlighted entities
highlighted_tokens = []

# Loop through each token and mark the entity tokens with highlighting
for token in entities:
    word = token["word"]
    entity_group = token["entity_group"]
    start_idx = token["start"]
    end_idx = token["end"]

    # Create a highlighted version of the token
    highlighted_token = f"{word} [{entity_group}]"

    # Add the highlighted token to the list
    highlighted_tokens.append(highlighted_token)

# Join the tokens to form the modified input text
highlighted_text = "\n".join(highlighted_tokens)

print(highlighted_text)


blood pressure [Diagnostic_procedure]
120 [Lab_value]
80 [Lab_value]
heart rate [Diagnostic_procedure]
72 [Lab_value]
temperature [Diagnostic_procedure]
98 [Lab_value]
6° [Lab_value]
respiratory rate [Diagnostic_procedure]
16 breaths [Lab_value]
over [Detailed_description]
the [Detailed_description]
follow [Clinical_event]
appointment [Clinical_event]
in [Date]
3 days [Date]


In [ ]:
input_text = " Name: John Doe, Age: 45, Blood Pressure: 120/80 mmHg, Heart Rate: 72 bpm, Temperature: 98.6°F, Respiratory Rate: 16 breaths/min. Diagnosis: The patient presents with symptoms of upper respiratory infection, including cough, nasal congestion, and low-grade fever. Suspected viral etiology. Treatment plan includes rest, hydration, and over-the-counter cold medications. Follow-up appointment scheduled in 3 days for reassessment.."
entities = pipe(input_text)

# Define color codes for each entity group
color_codes = {
    "Sign_symptom": "\033[91m",  # Red
    "Clinical_event": "\033[94m",  # Blue
    "Date": "\033[92m",  # Green
    "Diagnostic_procedure": "\033[95m",
    "Lab_value": "\033[96m"
}

# Create a list to store the tokens with highlighted entities
highlighted_tokens = []

# Loop through each token and mark the entity tokens with highlighting
for token in entities:
    word = token["word"]
    entity_group = token["entity_group"]

    # Determine the color code for the entity group
    color_code = color_codes.get(entity_group, "\033[0m")  # Default to reset color if entity group not found

    # Create a highlighted version of the entity group with color and entity group, while keeping word in default white
    highlighted_token = f"\033[0m{word} {color_code}[{entity_group}]\033[0m"

    # Add the highlighted token to the list
    highlighted_tokens.append(highlighted_token)

# Join the tokens to form the modified input text
highlighted_text = "\n".join(highlighted_tokens)

print(highlighted_text)


blood pressure [Diagnostic_procedure]
120 / 80 mm [Lab_value]
heart rate [Diagnostic_procedure]
72 bp [Lab_value]
temperature [Diagnostic_procedure]
98. 6° [Lab_value]
respiratory rate [Diagnostic_procedure]
16 breaths [Lab_value]
respiratory [Biological_structure]
follow [Clinical_event]
appointment [Clinical_event]
in 3 days [Duration]


In [ ]:
input_text = ""
entities = pipe(input_text)

# Define color codes for each entity group
color_codes = {
    "Sign_symptom": "\033[91m",  # Red
    "Clinical_event": "\033[94m",  # Blue
    "Date": "\033[92m",  # Green
    "Diagnostic_procedure": "\033[95m",
    "Lab_value": "\033[96m"
}

# Create a list to store the tokens with highlighted entities
highlighted_tokens = []

# Loop through each token and mark the entity tokens with highlighting
for token in entities:
    word = token["word"]
    entity_group = token["entity_group"]

    # Determine the color code for the entity group
    color_code = color_codes.get(entity_group, "\033[0m")  # Default to reset color if entity group not found

    # Create a highlighted version of the entity group with color and entity group, while keeping word in default white
    highlighted_token = f"\033[0m{word} {color_code}[{entity_group}]\033[0m"

    # Add the highlighted token to the list
    highlighted_tokens.append(highlighted_token)

# Join the tokens to form the modified input text
highlighted_text = "\n".join(highlighted_tokens)

print(highlighted_text)


In [ ]:
input_text = "Patient presented with symptoms of fever, cough, and shortness of breath, indicative of pneumonia. Upon examination, diagnosed with bacterial pneumonia and prescribed a course of antibiotics  resulting in symptom resolution within a week."
entities = pipe(input_text)

# Define color codes for each entity group
color_codes = {
    "Sign_symptom": "\033[91m",  # Red
    "Clinical_event": "\033[94m",  # Blue
    "Date": "\033[92m",  # Green
    "Diagnostic_procedure": "\033[95m",
    "Lab_value": "\033[96m"
}

# Create a list to store the tokens with highlighted entities
highlighted_tokens = []

# Loop through each token and mark the entity tokens with highlighting
for token in entities:
    word = token["word"]
    entity_group = token["entity_group"]

    # Determine the color code for the entity group
    color_code = color_codes.get(entity_group, "\033[0m")  # Default to reset color if entity group not found

    # Create a highlighted version of the entity group with color and entity group, while keeping word in default white
    highlighted_token = f"\033[0m{word} {color_code}[{entity_group}]\033[0m"

    # Add the highlighted token to the list
    highlighted_tokens.append(highlighted_token)

# Join the tokens to form the modified input text
highlighted_text = "\n".join(highlighted_tokens)

print(highlighted_text)


fever [Sign_symptom]
shortness of breath [Sign_symptom]
bacterial [Detailed_description]
pneumonia [Disease_disorder]
antibiotics [Medication]
symptom [Sign_symptom]


In [ ]:
input_text = "The patient presented with fever, cough, and shortness of breath, suggestive of a respiratory infection. Further evaluation revealed bilateral lung infiltrates on chest X-ray consistent with pneumonia."

# Define color codes for each entity group
color_codes = {
    "Sign_symptom": "\033[91m",  # Red
    "Clinical_event": "\033[94m",  # Blue
    "Date": "\033[92m",  # Green
    "Diagnostic_procedure": "\033[95m",
    "Lab_value": "\033[96m"
}

# Create a list to store the tokens with highlighted entities
highlighted_tokens = []

# Loop through each token and mark the entity tokens with highlighting
for token in entities:
    word = token["word"]
    entity_group = token["entity_group"]

    # Determine the color code for the entity group
    color_code = color_codes.get(entity_group, "\033[0m")  # Default to reset color if entity group not found

    # Create a highlighted version of the entity group with color and entity group, while keeping word in default white
    highlighted_token = f"\033[0m{word} {color_code}[{entity_group}]\033[0m"

    # Add the highlighted token to the list
    highlighted_tokens.append(highlighted_token)

# Join the tokens to form the modified input text
highlighted_text = "\n".join(highlighted_tokens)

print(highlighted_text)


fever [Sign_symptom]
shortness of breath [Sign_symptom]
bacterial [Detailed_description]
pneumonia [Disease_disorder]
antibiotics [Medication]
symptom [Sign_symptom]


In [ ]:
input_text = " Name: John Doe, Age: 45, Blood Pressure: 120/80 mmHg, Heart Rate: 72 bpm, Temperature: 98.6°F, Respiratory Rate: 16 breaths/min. Diagnosis: The patient presents with symptoms of upper respiratory infection, including cough, nasal congestion, and low-grade fever. Suspected viral etiology. Treatment plan includes rest, hydration, and over-the-counter cold medications. Follow-up appointment scheduled in 3 days for reassessment.."
entities = pipe(input_text)

# Define color codes for each entity group
color_codes = {
    "Sign_symptom": "\033[91m",  # Red
    "Clinical_event": "\033[94m",  # Blue
    "Date": "\033[92m",  # Green
    "Diagnostic_procedure": "\033[95m",
    "Lab_value": "\033[96m"
}

# Create a list to store the tokens with highlighted entities
highlighted_tokens = []

# Loop through each token and mark the entity tokens with highlighting
for token in entities:
    word = token["word"]
    entity_group = token["entity_group"]

    # Determine the color code for the entity group
    color_code = color_codes.get(entity_group, "\033[0m")  # Default to reset color if entity group not found

    # Create a highlighted version of the entity group with color and entity group, while keeping word in default white
    highlighted_token = f"\033[0m{word} {color_code}[{entity_group}]\033[0m"

    # Add the highlighted token to the list
    highlighted_tokens.append(highlighted_token)

# Join the tokens to form the modified input text
highlighted_text = "\n".join(highlighted_tokens)

print(highlighted_text)


blood pressure [Diagnostic_procedure]
120 / 80 mm [Lab_value]
heart rate [Diagnostic_procedure]
72 bp [Lab_value]
temperature [Diagnostic_procedure]
98. 6° [Lab_value]
respiratory rate [Diagnostic_procedure]
16 breaths [Lab_value]
respiratory [Biological_structure]
follow [Clinical_event]
appointment [Clinical_event]
in 3 days [Duration]


In [ ]:
input_text = "A 48 year-old female presented with vaginal bleeding and abnormal Pap smears. Upon diagnosis of invasive non-keratinizing SCC of the cervix, she underwent a radical hysterectomy with salpingo-oophorectomy which demonstrated positive spread to the pelvic lymph nodes and the parametrium. Pathological examination revealed that the tumor also extensively involved the lower uterine segment."
entities = pipe(input_text)

# Define color codes for each entity group
color_codes = {
    "Sign_symptom": "\033[91m",  # Red
    "Clinical_event": "\033[94m",  # Blue
    "Date": "\033[92m",  # Green
    "Diagnostic_procedure": "\033[95m",
    "Lab_value": "\033[96m"
}

# Create a list to store the tokens with highlighted entities
highlighted_tokens = []

# Loop through each token and mark the entity tokens with highlighting
for token in entities:
    word = token["word"]
    entity_group = token["entity_group"]

    # Determine the color code for the entity group
    color_code = color_codes.get(entity_group, "\033[0m")  # Default to reset color if entity group not found

    # Create a highlighted version of the entity group with color and entity group, while keeping word in default white
    highlighted_token = f"\033[0m{word} {color_code}[{entity_group}]\033[0m"

    # Add the highlighted token to the list
    highlighted_tokens.append(highlighted_token)

# Join the tokens to form the modified input text
highlighted_text = "\n".join(highlighted_tokens)

print(highlighted_text)


48 year - old [Age]
female [Sex]
presented [Clinical_event]
vaginal [Biological_structure]
bleeding [Sign_symptom]
abnormal [Lab_value]
pap [Diagnostic_procedure]
invasive [Detailed_description]
non [Detailed_description]
keratini [Detailed_description]
sc [Disease_disorder]
radical [Detailed_description]
h [Therapeutic_procedure]
##ysterectomy [Therapeutic_procedure]
sal [Therapeutic_procedure]
##pingo - oophorectomy [Therapeutic_procedure]
positive [Lab_value]
pelvic lymph nodes [Biological_structure]
para [Biological_structure]
path [Diagnostic_procedure]
examination [Diagnostic_procedure]
tumor [Coreference]
lower uterine [Biological_structure]


In [ ]:
input_text = ""
entities = pipe(input_text)

# Define color codes for each entity group
color_codes = {
    "Sign_symptom": "\033[91m",  # Red
    "Clinical_event": "\033[94m",  # Blue
    "Date": "\033[92m",  # Green
    "Diagnostic_procedure": "\033[95m",
    "Lab_value": "\033[96m"
}

# Create a list to store the tokens with highlighted entities
highlighted_tokens = []

# Loop through each token and mark the entity tokens with highlighting
for token in entities:
    word = token["word"]
    entity_group = token["entity_group"]

    # Determine the color code for the entity group
    color_code = color_codes.get(entity_group, "\033[0m")  # Default to reset color if entity group not found

    # Create a highlighted version of the entity group with color and entity group, while keeping word in default white
    highlighted_token = f"\033[0m{word} {color_code}[{entity_group}]\033[0m"

    # Add the highlighted token to the list
    highlighted_tokens.append(highlighted_token)

# Join the tokens to form the modified input text
highlighted_text = "\n".join(highlighted_tokens)

print(highlighted_text)
